In [2]:
%matplotlib inline

In [3]:
import numpy
import numpy as np
import matplotlib
from matplotlib import pyplot
from matplotlib import pyplot as plt

In [4]:
#### AREPOSNAP-UTIL packages ####
# in following packages some of the parameters might be useful to change. still have to figure out which

# I think areposnap.gadget is mostly to read in data and do some transformation with them... did not find
from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind


In [10]:
# written by Timo Halbesma (MPA); added some additional comments for better understanding
def eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[0,1,2,3,4],
                     haloid=0, galradfac=0.1, verbose=True):
    """ Method to eat an Auriga snapshot, given a level/halo_number/snapnr.
        Subfind has been executed 'on-the-fly', during the simulation run.

        @param level: level of the Auriga simulation (3=high, 4='normal' or 5=low).
            Level 3/5 only for halo 6, 16 and 24. See Grand+ 2017 for details.
            Careful when level != 4 because directories may have different names.
        @param halo_number: which Auriga galaxy? See Grand+ 2017 for details.
            Should be an integer in range(1, 31)
        @param snapnr: which snapshot number? This is an integer, in most cases
            in range(1, 128) depending on the number of timesteps of the run.
            The last snapshot would then be 127. Snapshots are written at a
            certain time, but careful because the variable called time is actually
            the cosmological expansion factor a = 1/(1+z). For example, snapnr=127
            has s.time = 1, which corresponds to a redshift of ~0. This makes sense
            because this is the last snapshot and the last snapshot is written at
            redshift zero
        @param snappath: full path to the level/halo directory that contains
            all of the simulation snapshots
        @param loadonlytype: which particle types should be loaded? This should
            be a list of integers. If I'm not mistaken, the options are:
            0 (gas), 1 (halo), 2 (disk), 3 (bulge), 4 (stars), 5 (black holes).
            So to get the dark matter: load particles 1/2/3. In zoom-simulations
            particletype 3 may be used for low-resolution particles in the outer
            regions and they should not be present in (and contaminating) the
            inner region. I'm not too sure of the latter though.
        @param haloid: the ID of the SubFind halo. In case you are interested
            in the main galaxy in the simulation run: set haloid to zero.
            This was a bit confusing to me at first because a zoom-simulation run
            of one Auriga galaxy is also referred to as 'halo', see halo_number.
        @param galradfac: the radius of the galaxy is often used to make cuts in
            the (star) particles. It seems that in general galrad is set to 10%
            of the virial radius R200 of the DM halo that the galaxy sits in. The
            disk does seem to 'end' at 0.1R200.
        @param verbose: boolean to print some information

        @return: two-tuple (s, sf) where s is an instance of the gadget_snapshot
            class, and sf is an instance of the subfind class. See Arepo-snap-util,
            gadget_snap.py respectively gadget_subfind.py """

    # Eat the subfind friend of friends output
    sf = load_subfind(snapnr, dir=snappath)

    # Eat the Gadget snapshot
    s = gadget_readsnap(snapnr, snappath=snappath, lazy_load=True,
        subfind=sf, loadonlytype=loadonlytype)
    s.subfind = sf

    # Sets s.(sub)halo. This allows selecting the halo, e.g. 0 (main 'Galaxy')
    s.calc_sf_indizes(s.subfind)
    # Note that selecting the halo now rotates the disk using the principal axis.
    # rotate_disk is a general switch which has to be set to True to rotate.
    # To then actually do the rotation, do_rotation has to be True as well.
    # Within rotate_disk there are three methods to handle the rotation. Choose
    # one of them, but see the select_halo method for details.
    s.select_halo( s.subfind, haloid=haloid, galradfac=galradfac,
        rotate_disk=True, use_principal_axis=True, euler_rotation=False,
        use_cold_gas_spin=False, do_rotation=True )

    # Sneak some more info into the s instance
    s.halo_number = halo_number
    s.level = level
    s.snapnr = snapnr
    s.haloid = haloid

    # This means that galrad is 10 % of R200 (200*rho_crit definition)
    s.galrad = galradfac * sf.data['frc2'][haloid]  # frc2 = Group_R_Crit200

    if verbose:
        print("\ngalrad  : {0}".format(s.galrad))
        print("redshift: {0}".format(s.redshift))
        print("time    : {0}".format(s.time))
        print("center  : {0}\n".format(s.center))

    return s, sf


In [12]:
def get_cartesian_vectors(s, sf, mask):
    x,  y,  z  = s.pos[::,2][mask], s.pos[::,1][mask], s.pos[::,0][mask]
    vx, vy, vz = s.vel[::,2][mask], s.vel[::,1][mask], s.vel[::,0][mask]

    # All three expressions for rxyz compute the same thing :-).
    # rxyz = numpy.sqrt( x**2 + y**2 + z**2)
    # rxyz = numpy.sqrt( (s.pos[mask]**2).sum(axis=1) )
    rxyz = s.r()[mask]

    # Both expressions for rxy compute the same thing :-).
    # rxy = numpy.sqrt( (s.pos[mask][:,1:]**2).sum(axis=1) )
    rxy = numpy.sqrt( x**2 + y**2 )

    return (x, y, z), (vx, vy, vz), rxyz, rxy



In [11]:
###### load several low res snapshots
level = 5
basedir = "/Users/smilanov/Desktop/Auriga/level{0}/".format(level)
j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo{0}_MHD/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(63,64,1):
        '''
        print("level   : {0}".format(level))
        print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print("basedir : {0}".format(basedir))
        print("halodir : {0}".format(halodir))
        print("snappath: {0}\n".format(snappath))
        '''
        s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath,
            haloid=0, galradfac=0.1, verbose=False) 

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        s.data['gmet'] = numpy.maximum( s.data['gmet'], 1e-40 )
        

       

[ 34.41684723  33.15987778  37.29312134]
Found 210233 stars.
Rotated pos.
Rotated bfld.
Rotated vel.


In [14]:
iall, = numpy.where( (s.halo == 0) )
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, iall)
ids = s.id[iall]
m = s.mass[iall]
ty = s.type[iall]

In [51]:
np.savetxt('/Users/smilanov/Documents/masterthesis/data/level_' + str(level) + '_halo_' + \
           str(halo_number) + '_snapshot_' + str(snapnr) + '_data.txt', \
           np.column_stack((ids, ty, x, y, z, vx, vy, vz, m)), \
           fmt = '%d %d %10.25f %10.25f %10.25f %10.20f %10.20f %10.20f %10.20f' , \
           header = 'IDs, type, x, y, z, vx, vy, vz, mass')